In [60]:
import pandas as pd
import numpy as np
import re
from string import ascii_letters, digits
import utils.cleaning_dicts
#import matplotlib
import os
import sys

In [61]:
%reload_ext autoreload
%autoreload 2
#%matplotlib inline

In [62]:
print(utils.cleaning_dicts.clean_team_pff)

{'arz': 'ari', 'blt': 'bal', 'clv': 'cle', 'hst': 'hou', 'la': 'lar', 'sd': 'lac', 'sl': 'lar'}


In [63]:
os.path.abspath(os.getcwd())
!ls

chromedriver			       other_data
create_modeling_data_sample_all.ipynb  pfr
create_modeling_data_sample.ipynb      README.md
create_player_pools_2022.ipynb	       rush_summary.csv
current_data			       scripts
fo_roll.csv			       spreads_data
historic_data			       tgs_modeling.ipynb
misc_files			       tgs_roll.csv
modeling_data			       Untitled.ipynb
modeling_file_week_4_DRAFT.csv.csv     utils
modeling.ipynb			       weather_scraper_current_year.py
notebooks


In [64]:
#cur_dir = os.path.abspath(os.getcwd())
cur_week_int = 6
cur_week_str = str(6)

### Read in and process all pff position datasets

In [65]:
####################################################################################
                ###   Read-in and clean all passing datasets ###
####################################################################################

passing_depth = pd.read_csv('./historic_data/pff_data/passing_depth_hist.csv')
passing_allowed_pressure = pd.read_csv('./historic_data/pff_data/passing_allowed_pressure_hist.csv')
passing_pressure = pd.read_csv('./historic_data/pff_data/passing_pressure_hist.csv')
passing_concept = pd.read_csv('./historic_data/pff_data/passing_concept_hist.csv')
time_in_pocket = pd.read_csv('./historic_data/pff_data/time_in_pocket_hist.csv')
passing_summ_conc = pd.read_csv('./historic_data/pff_data/passing_summ_conc_hist.csv')

passing_depth_new = pd.read_csv('./scripts/nfl_all/passing_depth_2022.csv')
passing_allowed_pressure_new = pd.read_csv('./scripts/nfl_all/passing_allowed_pressure_2022.csv')
passing_pressure_new = pd.read_csv('./scripts/nfl_all/passing_pressure_2022.csv')
passing_concept_new = pd.read_csv('./scripts/nfl_all/passing_concept_2022.csv')
time_in_pocket_new = pd.read_csv('./scripts/nfl_all/time_in_pocket_2022.csv')
passing_summ_conc_new = pd.read_csv('./scripts/nfl_all/passing_summ_conc_2022.csv')
                                 
passing_depth = pd.concat([passing_depth, passing_depth_new], axis=0).reset_index(drop=True)
passing_allowed_pressure = pd.concat([passing_allowed_pressure, passing_allowed_pressure_new], axis=0).reset_index(drop=True)
passing_pressure = pd.concat([passing_pressure, passing_pressure_new], axis=0).reset_index(drop=True)
passing_concept = pd.concat([passing_concept, passing_concept_new], axis=0).reset_index(drop=True)
time_in_pocket = pd.concat([time_in_pocket, time_in_pocket_new], axis=0).reset_index(drop=True)
passing_summ_conc = pd.concat([passing_summ_conc, passing_summ_conc_new], axis=0).reset_index(drop=True)
                                 

def drop_non_qbs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df=df[df['position'] == 'QB']
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df
    
passing_depth = drop_non_qbs(passing_depth)
passing_allowed_pressure = drop_non_qbs(passing_allowed_pressure)
passing_pressure = drop_non_qbs(passing_pressure)
passing_concept = drop_non_qbs(passing_concept)
time_in_pocket = drop_non_qbs(time_in_pocket)
passing_summ_conc = drop_non_qbs(passing_summ_conc)


passing_depth = passing_depth[passing_depth.columns.drop(list(passing_depth.filter(regex='left|right|center')))]

####################################################################################
				###   Read-in and clean all receiving datasets ### scripts/nfl_all
####################################################################################

rec_summ_conc = pd.read_csv('./historic_data/pff_data/rec_summ_conc_hist.csv')
receiving_concept = pd.read_csv('./historic_data/pff_data/receiving_concept_hist.csv')
receiving_depth = pd.read_csv('./historic_data/pff_data/receiving_depth_hist.csv')
receiving_scheme = pd.read_csv('./historic_data/pff_data/receiving_scheme_hist.csv')
                                 
rec_summ_conc_new = pd.read_csv('./scripts/nfl_all/rec_summ_conc_2022.csv')
receiving_concept_new = pd.read_csv('./scripts/nfl_all/receiving_concept_2022.csv')
receiving_depth_new = pd.read_csv('./scripts/nfl_all/receiving_depth_2022.csv')
receiving_scheme_new = pd.read_csv('./scripts/nfl_all/receiving_scheme_2022.csv')
                                 
rec_summ_conc = pd.concat([rec_summ_conc, rec_summ_conc_new], axis=0).reset_index(drop=True)
receiving_concept = pd.concat([receiving_concept, receiving_concept_new], axis=0).reset_index(drop=True)
receiving_depth = pd.concat([receiving_depth, receiving_depth_new], axis=0).reset_index(drop=True)
receiving_scheme = pd.concat([receiving_scheme, receiving_scheme_new], axis=0).reset_index(drop=True)                                 

def drop_non_recs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df= df[df.position.str.match('WR|TE|HB|FB')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df

rec_summ_conc = drop_non_recs(rec_summ_conc)
receiving_concept = drop_non_recs(receiving_concept)
receiving_depth = drop_non_recs(receiving_depth)
receiving_scheme = drop_non_recs(receiving_scheme)


####################################################################################
				###   Read-in and clean all rushing datasets ###
####################################################################################

rush_summ_conc = pd.read_csv('./historic_data/pff_data/rush_summ_conc_hist.csv')
rush_summ_conc_new = pd.read_csv('./scripts/nfl_all/rush_summ_conc_2022.csv')                                 
                                 
rush_summ_conc = pd.concat([rush_summ_conc, rush_summ_conc_new], axis=0)
 

def drop_non_rbs(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df= df[df.position.str.match('WR|HB|FB|QB')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df

rush_summ_conc = drop_non_rbs(rush_summ_conc)


####################################################################################
				###   Read-in and clean all blocking datasets ###
####################################################################################


block_summ_conc = pd.read_csv('./historic_data/pff_data/block_summ_conc_hist.csv')
offense_pass_blocking = pd.read_csv('./historic_data/pff_data/offense_pass_blocking_hist.csv')
offense_run_blocking = pd.read_csv('./historic_data/pff_data/offense_run_blocking_hist.csv')
                                 
block_summ_conc_new = pd.read_csv('./scripts/nfl_all/block_summ_conc_2022.csv')
offense_pass_blocking_new = pd.read_csv('./scripts/nfl_all/offense_pass_blocking_2022.csv')
offense_run_blocking_new = pd.read_csv('./scripts/nfl_all/offense_run_blocking_2022.csv')                                 

block_summ_conc = pd.concat([block_summ_conc, block_summ_conc_new], axis=0).reset_index(drop=True)
offense_pass_blocking = pd.concat([offense_pass_blocking, offense_pass_blocking_new], axis=0).reset_index(drop=True)
offense_run_blocking = pd.concat([offense_run_blocking, offense_run_blocking_new], axis=0).reset_index(drop=True)

def drop_non_ols(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df = df[df['position'].notna()]
    df= df[df.position.str.match('T|C|G|TE')]
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df


block_summ_conc	= drop_non_ols(block_summ_conc)
offense_pass_blocking = drop_non_ols(offense_pass_blocking)
offense_run_blocking = drop_non_ols(offense_run_blocking)



####################################################################################
				###   Read-in and clean all defensive datasets ###
####################################################################################

def_summ_conc = pd.read_csv('./historic_data/pff_data/def_summ_conc_hist.csv')
pass_rush_summary = pd.read_csv('./historic_data/pff_data/pass_rush_summary_hist.csv')
run_defense_summary = pd.read_csv('./historic_data/pff_data/run_defense_summary_hist.csv')
defense_coverage_scheme = pd.read_csv('./historic_data/pff_data/defense_coverage_scheme_hist.csv')
defense_coverage_summary = pd.read_csv('./historic_data/pff_data/defense_coverage_summary_hist.csv')
slot_coverage = pd.read_csv('./historic_data/pff_data/slot_coverage_hist.csv')
                                 
def_summ_conc_new = pd.read_csv('./scripts/nfl_all/def_summ_conc_2022.csv')
pass_rush_summary_new = pd.read_csv('./scripts/nfl_all/pass_rush_summary_2022.csv')
run_defense_summary_new = pd.read_csv('./scripts/nfl_all/run_defense_summary_2022.csv')
defense_coverage_scheme_new = pd.read_csv('./scripts/nfl_all/defense_coverage_scheme_2022.csv')
defense_coverage_summary_new = pd.read_csv('./scripts/nfl_all/defense_coverage_summary_2022.csv')
slot_coverage_new = pd.read_csv('./scripts/nfl_all/slot_coverage_2022.csv')

def_summ_conc = pd.concat([def_summ_conc, def_summ_conc_new], axis=0).reset_index(drop=True)
pass_rush_summary = pd.concat([pass_rush_summary, pass_rush_summary_new], axis=0).reset_index(drop=True)
run_defense_summary = pd.concat([run_defense_summary, run_defense_summary_new], axis=0).reset_index(drop=True)
defense_coverage_scheme = pd.concat([defense_coverage_scheme, defense_coverage_scheme_new], axis=0).reset_index(drop=True)
defense_coverage_summary = pd.concat([defense_coverage_summary, defense_coverage_summary_new], axis=0).reset_index(drop=True)
slot_coverage = pd.concat([slot_coverage, slot_coverage_new], axis=0).reset_index(drop=True)
                                 
def drop_non_def(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df

def_summ_conc = drop_non_def(def_summ_conc)
pass_rush_summary = drop_non_def(pass_rush_summary)
run_defense_summary = drop_non_def(run_defense_summary)
defense_coverage_scheme = drop_non_def(defense_coverage_scheme)
defense_coverage_summary = drop_non_def(defense_coverage_summary)
slot_coverage = drop_non_def(slot_coverage)

def_summ_conc=def_summ_conc[def_summ_conc['position'].isin(["ed","lb","di","s","cb"])]
pass_rush_summary=pass_rush_summary[pass_rush_summary['position'].isin(["ed","lb","di","s"])]
run_defense_summary=run_defense_summary[run_defense_summary['position'].isin(["ed","lb","di","s","cb"])]
defense_coverage_scheme=defense_coverage_scheme[defense_coverage_scheme['position'].isin(["lb","s","cb"])]
defense_coverage_summary=defense_coverage_summary[defense_coverage_summary['position'].isin(["lb","s","cb"])]
slot_coverage=slot_coverage[slot_coverage['position'].isin(["lb","s","cb"])]

####################################################################################
				###   Read-in and clean all special teams datasets ###
####################################################################################	

st_kickers = pd.read_csv('./historic_data/pff_data/st_kickers_hist.csv')
st_punters = pd.read_csv('./historic_data/pff_data/st_punters_hist.csv')

st_kickers_new = pd.read_csv('./scripts/nfl_all/st_kickers_2022.csv')
st_punters_new = pd.read_csv('./scripts/nfl_all/st_punters_2022.csv')                                 
                                 
                                 
st_kickers = pd.concat([st_kickers, st_kickers_new], axis=0).reset_index(drop=True)
st_punters = pd.concat([st_punters, st_punters_new], axis=0).reset_index(drop=True)
                                 
def clean_spec(df):
    df=df.rename(columns={"player_id": "numeric_id"})
    df['position']=df['position'].astype(str).str.lower()
    df['team_name']=df['team_name'].astype(str).str.lower()       
    df['player']=df['player'].str.replace('[^a-zA-Z0-9]', '').str.lower()
    df['team_name']=df['team_name'].str.lower()
    df['team_name']=df['team_name'].replace("oak","lv")
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
        ##  pass team name through dictionary to clean ##
    df['team_name'] = df['team_name'].map(utils.cleaning_dicts.clean_team_pff).fillna(df['team_name'])
    df['position'] = df['position'].map(utils.cleaning_dicts.pos_dict).fillna(df['position'])

    
    df.insert(0, "p_id", (df['player']+'_'+df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(2, "player_team_id", (df['player']+'_'+df['team_name']+'_'+df['year']))
    df.insert(3, "team_id_impute", (df['team_name']+'_'+df['year']))
    return df

st_kickers =clean_spec(st_kickers)
st_punters = clean_spec(st_punters)



####################################################################################
####################################################################################
####################################################################################

/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:83: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:121: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:163: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:215: FutureWarning: The default value of regex will change from True to False in a future version.
/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:263: FutureWarn

In [66]:
passing_depth.columns

Index(['p_id', 'unique_team_id', 'player_team_id', 'team_id_impute', 'player',
       'numeric_id', 'position', 'team_name', 'player_game_count',
       'base_attempts',
       ...
       'short_scrambles', 'short_spikes', 'short_thrown_aways',
       'short_touchdowns', 'short_turnover_worthy_plays', 'short_twp_rate',
       'short_yards', 'short_ypa', 'week', 'year'],
      dtype='object', length=144)

In [67]:
%%time

def impute(df):
    df = df.apply(pd.to_numeric, errors='ignore')
    df.reset_index(inplace=True, drop=True)
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    df[num_cols]= df.groupby(df['team_id_impute'])[num_cols].fillna(df.mean()).reset_index(level=0, drop=True)
    return df

passing_depth = impute(passing_depth)
passing_allowed_pressure = impute(passing_allowed_pressure)
passing_pressure = impute(passing_pressure)
passing_concept = impute(passing_concept)
time_in_pocket = impute(time_in_pocket)
passing_summ_conc = impute(passing_summ_conc)

rec_summ_conc = impute(rec_summ_conc)
receiving_concept = impute(receiving_concept)
receiving_depth = impute(receiving_depth)
receiving_scheme = impute(receiving_scheme)

rush_summ_conc = impute(rush_summ_conc)

block_summ_conc = impute(block_summ_conc)
offense_pass_blocking = impute(offense_pass_blocking)
offense_run_blocking = impute(offense_run_blocking)

def_summ_conc = impute(def_summ_conc)
pass_rush_summary = impute(pass_rush_summary)
run_defense_summary = impute(run_defense_summary)
defense_coverage_scheme = impute(defense_coverage_scheme)
defense_coverage_summary = impute(defense_coverage_summary)
slot_coverage = impute(slot_coverage)

st_kickers = impute(st_kickers)
st_punters = impute(st_punters)

/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """


CPU times: user 1min 22s, sys: 200 ms, total: 1min 22s
Wall time: 1min 22s


### Add prefixes to all columns.  Creating column names structured as "source-dataset_column-name"

In [68]:
####################################################################################
								###   add prefixes ###
####################################################################################	

def create_prefix(prefix=None, df=None):
    id = df[['p_id','player_team_id','unique_team_id','team_id_impute','player','numeric_id','position','team_name','year','week']]
    temp = df.drop(['p_id','player','player_team_id','unique_team_id','player','team_id_impute','numeric_id','position','team_name','unique_team_id','numeric_id','position','team_name','year','week'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id

def id_prefix(prefix=None, df=None):
    id = df[['p_id','player','player_team_id','unique_team_id','team_id_impute','numeric_id','position','team_name','year','week']]
    temp = df.drop(['p_id','player','player_team_id','unique_team_id','team_id_impute','numeric_id','position','team_name','year','week'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id

passing_summ_conc = id_prefix(prefix="pass_summary_", df=passing_summ_conc)
rush_summ_conc = id_prefix(prefix="rush_summary_", df=rush_summ_conc)
rec_summ_conc = id_prefix(prefix="rec_summary_", df=rec_summ_conc)
block_summ_conc = id_prefix(prefix="block_summary_", df=block_summ_conc)
def_summ_conc = id_prefix(prefix="def_summary_", df=def_summ_conc)
st_kickers = id_prefix(prefix="kicking_", df=st_kickers)
st_punters = id_prefix(prefix="punting_", df=st_punters)


passing_depth = create_prefix(prefix="pass_depth_", df=passing_depth)
passing_allowed_pressure = create_prefix(prefix="pressure_source_", df=passing_allowed_pressure)
passing_pressure = create_prefix(prefix="pass_under_pressure_", df=passing_pressure)
passing_concept = create_prefix(prefix="pass_concept_", df=passing_concept)
time_in_pocket = create_prefix(prefix="pass_time_", df=time_in_pocket)


receiving_concept = create_prefix(prefix="rec_concept_", df=receiving_concept)
receiving_depth = create_prefix(prefix="rec_depth_", df=receiving_depth)
receiving_scheme = create_prefix(prefix="rec_scheme_", df=receiving_scheme)

offense_pass_blocking = create_prefix(prefix="pass_block_", df=offense_pass_blocking)
offense_run_blocking = create_prefix(prefix="run_block_", df=offense_run_blocking)


pass_rush_summary = create_prefix(prefix="pass_rush_", df=pass_rush_summary)
run_defense_summary = create_prefix(prefix="run_defense_", df=run_defense_summary)
defense_coverage_scheme = create_prefix(prefix="def_coverage_scheme_", df=defense_coverage_scheme)
defense_coverage_summary = create_prefix(prefix="def_coverage_summary_", df=defense_coverage_summary)
slot_coverage = create_prefix(prefix="def_slot_coverage_", df=slot_coverage)



### Read in weather data and clean raiders name - merged onto spreads data below ###

In [69]:
### read in weather data###
weather = pd.read_csv('./current_data/week_'+cur_week_str+'/weather_hist_all.csv')

def raiders(df):
    if 'oak' in str(df.away_matchup_id) and '2020' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    if 'oak' in str(df.away_matchup_id) and '2021' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    if 'oak' in str(df.away_matchup_id) and '2022' in str(df.away_matchup_id):
        return df.away_matchup_id.replace("oak","lv")
    else:
        return df.away_matchup_id
weather['away_matchup_id'] = weather.apply(lambda df: raiders(df), axis=1)

### Create spreads data ###

In [70]:
####################################################################################
				###   spreads data cleaning and engineering ###
####################################################################################

spreads = pd.read_csv('./spreads_data/spreads.csv')

new_acc = {'oak':'lv',
          'sd':'lac',
          'stl':'lar'}  

spreads['team_home_abb'] = spreads['team_home_abb'].map(new_acc).fillna(spreads['team_home_abb'])
spreads['away_team_abb'] = spreads['away_team_abb'].map(new_acc).fillna(spreads['away_team_abb']) 

spreads = spreads[spreads['schedule_season']>=2014]
spreads = spreads[['schedule_season','schedule_week','team_home_abb','score_home','score_away','away_team_abb','team_favorite_id','spread_favorite','over_under_line','starting_spread', 'Total Score Open',
       'fav_team_open', 'fav_team_cur', 'remain_fav', 'spread_movement','ou_movement', 'strong_movement', 'fav_team_stronger']]
spreads['team_home_abb'] = spreads['team_home_abb'].astype(str)
spreads['team_favorite_id'] = spreads['team_favorite_id'].astype(str)
spreads['over_under_line'] = spreads['over_under_line'].astype(float)


def fav_spread(nData):
    if nData['team_home_abb'] == nData['team_favorite_id']:
        return nData['spread_favorite']
    elif nData['away_team_abb'] == nData['team_favorite_id']:
        return nData['spread_favorite']
    else:
        pass
spreads['fav_spread'] = spreads.apply(lambda nData: fav_spread(nData), axis=1)

def nonfav_spread(nData):
    if nData['team_home_abb'] != nData['team_favorite_id']:
        return nData['team_home_abb']
    elif nData['away_team_abb'] != nData['team_favorite_id']:
        return nData['away_team_abb']
    else:
        pass
spreads['team_notfav_id'] = spreads.apply(lambda nData: nonfav_spread(nData), axis=1)

def cover_or_not(nData):    
    if nData['team_home_abb'] == nData['team_favorite_id']:
        if ((nData['score_home']-nData['score_away']))+nData['spread_favorite'] > 0:
            return 'Cover'
        elif ((nData['score_home']-nData['score_away']))+nData['spread_favorite'] == 0:            
            return 'Push'       
        else:            
            return 'No Cover'
    elif nData['away_team_abb'] == nData['team_favorite_id']:        
        if ((nData['score_away']-nData['score_home']))+nData['spread_favorite'] > 0:            
            return 'Cover'        
        elif ((nData['score_away']-nData['score_home']))+nData['spread_favorite'] == 0:            
            return 'Push'        
        else:            
            return 'No Cover'
spreads['fav_cover'] = spreads.apply(lambda nData: cover_or_not(nData), axis=1)

def OU_or_not(nData):    
    if (nData['score_home']+nData['score_away']) > nData['over_under_line']:        
        return 'Over'    
    elif (nData['score_home']-nData['score_away']) == nData['over_under_line']:        
        return 'Push'    
    else:        
        return 'Under'
spreads['over_under_result'] = spreads.apply(lambda nData: OU_or_not(nData), axis=1)



spreads['schedule_season'] = spreads['schedule_season'].apply(int)    
spreads['schedule_week'] = spreads['schedule_week'].apply(int)  
data = spreads.sort_values(by=["team_home_abb","schedule_season","schedule_week"], ascending=[True, True, True])

def clean_spreads(df):
    ##  basic scrubbing to clean data ##    
    df['schedule_season'] = df['schedule_season'].apply(str)    
    df['schedule_week'] = df['schedule_week'].apply(str)        
    df=df.apply(lambda x: x.astype(str).str.lower())    
    #df['schedule_week']=df['schedule_week'].astype(str).str[:-2].astype(object)    
    #df['schedule_season'] = df['schedule_season'].astype(str).str[:-2].astype(object)        
    
    ##  create our unique ids  ##
    df.insert(0, "home_matchup_id", (df['team_home_abb']+'vs'+df['away_team_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(1, "away_matchup_id", (df['away_team_abb']+'@'+df['team_home_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(2, "home_id", (df['team_home_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    df.insert(3, "away_id", (df['away_team_abb']+'_'+df['schedule_season']+'_'+df['schedule_week']))
    return df
    
data = clean_spreads(data)

data = pd.merge(data, weather, on='away_matchup_id', how='left')


sh = data
sa = data

sh = sh.rename(columns={'home_id':'team_id'})
sh.drop('away_id', axis=1, inplace=True)

sa = sa.rename(columns={'away_id':'team_id'})
sa.drop('home_id', axis=1, inplace=True)

spread_comb = pd.concat([sh, sa], axis=0)
spread_comb['team_abb'] = spread_comb['team_id'].astype(str).str[:3]
spread_comb['team_abb'] = spread_comb['team_abb'].str.replace("_","")

def hora1(nData):
    if nData['team_favorite_id'] == nData['team_home_abb']:
        return 1
    elif nData['team_notfav_id'] == nData['team_home_abb']:
        return 1
    else:
        return 0
spread_comb['homeoraway'] = spread_comb.apply(lambda nData: hora1(nData), axis=1)

def hora(nData):
    if nData['team_favorite_id'] == nData['away_team_abb']:
        return 1
    else:
        return 0
spread_comb['fav_homeoraway'] = spread_comb.apply(lambda nData: hora(nData), axis=1)
#sh['fav_homeoraway'] = sh.apply(lambda nData: hora(nData), axis=1)

def ws(nData):
    if (nData['fav_homeoraway'] == 0) & (nData['fav_cover'] == 'cover'):
        return 1
    elif (nData['fav_homeoraway'] == 1) & (nData['fav_cover'] == 'no cover'):
        return 1
    else:
        return 0

def ls(nData):    
    if (nData['fav_homeoraway'] == 0) & (nData['fav_cover'] == 'no cover'):
        return 1
    elif (nData['fav_homeoraway'] == 1) & (nData['fav_cover'] == 'cover'):
        return 1
    else:
        return 0

spread_comb['ats_w'] = spread_comb.apply(lambda nData: ws(nData), axis=1)
spread_comb['ats_l'] = spread_comb.apply(lambda nData: ls(nData), axis=1)
spread_comb.head(n=5).T

,0,1,2,3,4
home_matchup_id,arivslac_2014_1,arivssf_2014_3,arivswas_2014_6,arivsphi_2014_8,arivslar_2014_10
away_matchup_id,lac@ari_2014_1,sf@ari_2014_3,was@ari_2014_6,phi@ari_2014_8,lar@ari_2014_10
team_id,ari_2014_1,ari_2014_3,ari_2014_6,ari_2014_8,ari_2014_10
schedule_season,2014,2014,2014,2014,2014
schedule_week,1,3,6,8,10
team_home_abb,ari,ari,ari,ari,ari
score_home,18,23,30,24,31
score_away,17,14,20,20,14
away_team_abb,lac,sf,was,phi,lar
team_favorite_id,ari,sf,ari,ari,ari


## Read in historic weekly football outsiders data.  Clean raiders name and computer rolling mean###

In [71]:


def rolling_fo(data=None, roll_value=None, roll_type=None):
    
    """
        Args:
        data: input pandas dataframe to be rolled
        roll_value: input the number, default is three ## we will need to modify the function if we want more ##
        roll_type: 'mean','std', or 'var' are the only options at the point
        ## assign mean for a given team & year as opposed to the entire dataset
   
    """
    
    data = data.sort_values(by=["team","year","week"], ascending=[True, True, True])
    #data=data.fillna(data.mean())
    num_cols = ['total_dvoa', 'off_dvoa','off_pass_dvoa', 'off_rush_dvoa', 'def_dvoa', 'def_pass_dvoa','def_rush_dvoa', 'special_teams_dvoa']
    ids = data[['team_id', 'year', 'team', 'week', 'opp']].reset_index(drop=True)
   
    if roll_type == 'mean':
        roll3 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll2 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).mean())
        roll1 = data.groupby(['team','year'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).mean())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    return df

In [72]:
	##Create the current weeks fo team_ids/rows to roll into##
fo_data = pd.read_csv("./current_data/week_"+cur_week_str+"/fo_weekly_update.csv")

prev = str(cur_week_int-1)
fo_data_new = fo_data[~fo_data['week'].isnull()]
fo_data_new=fo_data_new.drop_duplicates(subset=['team','year'], keep='last').assign(week=cur_week_str)
fo_data_new['team_id']=fo_data_new['team_id'].str.replace("2022_"+prev, str("2022_"+cur_week_str))

fo_data_new = fo_data_new.sort_values(by=["team","week"], ascending=[True, False])
fo_data_new[fo_data_new.columns[4:]] = np.nan
fo_data_new.head().T

,24,159,84,59,4
team_id,ari_2022_6,atl_2022_6,bal_2022_6,buf_2022_6,car_2022_6
year,2022,2022,2022,2022,2022
team,ari,atl,bal,buf,car
week,6,6,6,6,6
opp,NaN,NaN,NaN,NaN,NaN
total_dvoa,NaN,NaN,NaN,NaN,NaN
off_dvoa,NaN,NaN,NaN,NaN,NaN
off_pass_dvoa,NaN,NaN,NaN,NaN,NaN
off_rush_dvoa,NaN,NaN,NaN,NaN,NaN
def_dvoa,NaN,NaN,NaN,NaN,NaN


In [73]:
fo_data_2022 = pd.read_csv("./historic_data/fo_data/fo_weekly_hist.csv")
fo = pd.concat([fo_data_2022, fo_data, fo_data_new], axis=0)

fo['team'] = fo['team'].map(new_acc).fillna(fo['team'])
fo['opp'] = fo['opp'].map(new_acc).fillna(fo['opp']) 

fo['team'] = fo['team'].map(utils.cleaning_dicts.clean_team_fo).fillna(fo['team'])
fo['opp'] = fo['opp'].map(utils.cleaning_dicts.clean_team_fo).fillna(fo['opp'])

##combine our current season fo data with the new week 4 rows we just made##
fo_roll = rolling_fo(data=fo, roll_value=3, roll_type='mean')
fo_roll.head().T

,0,1,2,3,4
team_id,ari_2014_1,ari_2014_2,ari_2014_3,ari_2014_5,ari_2014_6
year,2014,2014,2014,2014,2014
team,ari,ari,ari,ari,ari
week,1,2,3,5,6
opp,lac,nyg,sf,den,was
total_dvoa,NaN,0.59,0.625,0.613333,0.56
off_dvoa,NaN,0.53,0.505,0.55,0.533333
off_pass_dvoa,NaN,0.53,0.45,0.516667,0.486667
off_rush_dvoa,NaN,0.51,0.57,0.55,0.56
def_dvoa,NaN,0.61,0.56,0.49,0.406667


### PFF team_game_summaries (tgs) clean and prepare for moving averages (if wanted)

In [74]:
tgs_new_week = pd.read_csv("./current_data/week_"+cur_week_str+"/team_game_summaries_w"+cur_week_str+".csv")

tgs_new_week = tgs_new_week[~tgs_new_week['week'].isnull()]
tgs_new_week=tgs_new_week.drop_duplicates(subset=['team','year'], keep='last').assign(week=cur_week_str)

tgs_new_week['team_name'] = tgs_new_week['team'].map(utils.cleaning_dicts.clean_team_pff_full).fillna(tgs_new_week['team'])
tgs_new_week['opponent_name'] = tgs_new_week['opponent'].map(utils.cleaning_dicts.clean_team_pff_opp).fillna(tgs_new_week['opponent'])

tgs_new_week['home_or_away']=tgs_new_week['home_or_away'].astype(str)

def home_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['opponent_name']
    else:
        return nData['team_name']

tgs_new_week['home_team'] = tgs_new_week.apply(lambda nData: home_team(nData), axis=1)

def away_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['team_name']
    else:
        return nData['opponent_name']
    
tgs_new_week['away_team'] = tgs_new_week.apply(lambda nData: away_team(nData), axis=1)

def clean_pff_team_summ(df):
##  basic scrubbing to clean data ##
    
    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df['home_or_away']=np.where(df['home_or_away'] == "@", 1, 0)
    df['wl_int'] = np.where(df['wl'] == "W", 1, 0)
    df=df.replace('-','', regex=True)
    df=df.replace(' ','', regex=True)


    ##  create our unique ids  ##
    df.insert(0, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "team_id_impute", (df['team_name']+'_'+df['year']))
    df.insert(2, "opponent_id", (df['opponent_name']+'_'+df['year']+'_'+df['week']))
    df.insert(3, "home_matchup_id", (df['home_team']+'vs'+df['away_team']+'_'+df['year']+'_'+df['week']))
    df.insert(4, "team_abb", (df['team_name']))

    return df

tgs_new_week = clean_pff_team_summ(tgs_new_week)
tgs_new_week['wl_int'] = ''
tgs_new_week = tgs_new_week.sort_values(by=["team_name","week"], ascending=[True, False])

In [75]:
tgs_data_2022 = pd.read_csv("./historic_data/pff_data/team_game_summaries_historic.csv")
tgs_data_cur = pd.read_csv("./current_data/week_"+cur_week_str+"/team_game_summaries_w"+cur_week_str+".csv")
tgs = pd.concat([tgs_data_2022, tgs_data_cur], axis=0)

tgs = tgs[tgs['year'] >= 2014]


tgs['team_name'] = tgs['team'].map(utils.cleaning_dicts.clean_team_pff_full).fillna(tgs['team'])
tgs['opponent_name'] = tgs['opponent'].map(utils.cleaning_dicts.clean_team_pff_opp).fillna(tgs['opponent'])

##adding just incase accronyms have changed
tgs['team_name'] = tgs['team_name'].map(new_acc).fillna(tgs['team_name'])
tgs['opponent_name'] = tgs['opponent_name'].map(new_acc).fillna(tgs['opponent_name']) 

tgs['home_or_away']=tgs['home_or_away'].astype(str)

def home_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['opponent_name']
    else:
        return nData['team_name']

tgs['home_team'] = tgs.apply(lambda nData: home_team(nData), axis=1)

def away_team(nData):
    if str('@') in nData['home_or_away']:
        return nData['team_name']
    else:
        return nData['opponent_name']
    
tgs['away_team'] = tgs.apply(lambda nData: away_team(nData), axis=1)

def clean_pff_team_summ(df):
##  basic scrubbing to clean data ##

    df['year'] = df['year'].astype(str)
    df['week'] = df['week'].astype(str)
    df['home_or_away']=np.where(df['home_or_away'] == "@", 1, 0)
    df['wl_int'] = np.where(df['wl'] == "W", 1, 0)
    df=df.replace('-','', regex=True)
    df=df.replace(' ','', regex=True)


    ##  create our unique ids  ##
    df.insert(0, "unique_team_id", (df['team_name']+'_'+df['year']+'_'+df['week']))
    df.insert(1, "team_id_impute", (df['team_name']+'_'+df['year']))
    df.insert(2, "opponent_id", (df['opponent_name']+'_'+df['year']+'_'+df['week']))
    df.insert(3, "home_matchup_id", (df['home_team']+'vs'+df['away_team']+'_'+df['year']+'_'+df['week']))
    
    ##Impute missing special teams data added after 2014##
    df = df.apply(pd.to_numeric, errors='ignore')
    df.reset_index(inplace=True, drop=True)
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    df[num_cols]= df.groupby(df['team_name'])[num_cols].fillna(df.mean()).reset_index(level=0, drop=True)
   
    return df


   
tgs_clean = clean_pff_team_summ(tgs)


tgs_clean = pd.concat([tgs_clean, tgs_new_week], axis=0)
tgs_clean['year']=tgs_clean['year'].apply(int)
tgs_clean['week']=tgs_clean['week'].apply(int)
tgs_clean['special_teams']=tgs_clean['special_teams'].apply(float)
tgs_clean = tgs_clean.sort_values(by=["team_abb","year","week"], ascending=[True, True, True])

tgs_clean = tgs_clean[['unique_team_id','team_id_impute', 'home_matchup_id','opponent_id','wl','pf','pa','team_name','opponent_name','year','week','overall_performance', 'offense', 'pass',
       'pass_blocking', 'receiving', 'rushing', 'run_blocking', 'defense',
       'rush_defense', 'tackling', 'pass_rush', 'coverage', 'special_teams']]



/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:54: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


In [76]:
tgs_clean.head()

,unique_team_id,team_id_impute,home_matchup_id,opponent_id,wl,pf,pa,team_name,opponent_name,year,...,pass_blocking,receiving,rushing,run_blocking,defense,rush_defense,tackling,pass_rush,coverage,special_teams
4,ari_2022_6,ari_2022,arivsphi_2022_6,phi_2022_6,L,17,20,ari,phi,2022,...,44.4,74.1,70.1,58.9,61.9,53.9,67.4,60.0,65.2,83.9
9,atl_2022_6,atl_2022,tbvsatl_2022_6,tb_2022_6,L,15,21,atl,tb,2022,...,59.7,65.5,73.6,63.9,57.2,65.2,51.5,55.5,53.4,74.3
14,bal_2022_6,bal_2022,balvscin_2022_6,cin_2022_6,W,19,17,bal,cin,2022,...,87.5,65.9,69.5,81.1,68.5,74.2,59.7,67.2,62.8,61.8
19,buf_2022_6,buf_2022,bufvspit_2022_6,pit_2022_6,W,38,3,buf,pit,2022,...,79.7,78.5,86.7,60.6,73.1,67.7,64.7,71.9,70.3,73.8
24,car_2022_6,car_2022,carvssf_2022_6,sf_2022_6,L,15,37,car,sf,2022,...,68.1,62.5,59.1,72.6,57.7,60.5,31.2,67.3,48.1,90.4


### Lets create a simple PFF team_game_summary dataset for modeling with averages across all games for a full season

In [77]:
def rolling_tgs(data=None, roll_value=None, roll_type=None):
    
    """
        Args:
        data: input pandas dataframe to be rolled
        roll_value: input the number, default is three ## we will need to modify the function if we want more ##
        roll_type: 'mean','std', or 'var' are the only options at the point
        ## assign mean for a given team & year as opposed to the entire dataset
   
    """
    
    data = data.sort_values(by=["team_name","year","week"], ascending=[True, True, True])
    num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    ids = pd.DataFrame(data.select_dtypes(exclude=[np.number])).reset_index(drop=True)
   
    if roll_type == 'mean':
        roll3 = data.groupby(data['team_id_impute'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll2 = data.groupby(data['team_id_impute'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).mean())
        roll1 = data.groupby(data['team_id_impute'])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).mean())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    return df
        
tgs_roll = rolling_tgs(data=tgs_clean, roll_value=3, roll_type='mean')

tgs_roll.drop(['year', 'week', 
                'team_id_impute','team_name'], axis=1, inplace=True)

tgs_roll = tgs_roll.add_suffix('_tgs_pff')
tgs_roll.rename(columns={'unique_team_id_tgs_pff':'unique_team_id'}, inplace=True)

In [78]:
passing_depth_new = pd.read_csv("./current_data/week_"+cur_week_str+"/passing_depth_new_pp_w"+cur_week_str+".csv")
passing_allowed_pressure_new = pd.read_csv('./current_data/week_'+cur_week_str+'/passing_allowed_pressure_new_pp_w'+cur_week_str+".csv")
passing_pressure_new = pd.read_csv('./current_data/week_'+cur_week_str+'/passing_pressure_new_pp_w'+cur_week_str+".csv")
passing_concept_new = pd.read_csv('./current_data/week_'+cur_week_str+'/passing_concept_new_pp_w'+cur_week_str+".csv")
time_in_pocket_new = pd.read_csv('./current_data/week_'+cur_week_str+'/time_in_pocket_new_pp_w'+cur_week_str+".csv")
passing_summ_conc_new = pd.read_csv('./current_data/week_'+cur_week_str+'/passing_summ_conc_new_pp_w'+cur_week_str+".csv")


rec_summ_conc_new = pd.read_csv('./current_data/week_'+cur_week_str+'/rec_summ_conc_pp_w'+cur_week_str+".csv")
receiving_concept_new = pd.read_csv('./current_data/week_'+cur_week_str+'/receiving_concept_pp_w'+cur_week_str+".csv")
receiving_depth_new = pd.read_csv('./current_data/week_'+cur_week_str+'/receiving_depth_pp_w'+cur_week_str+".csv")
receiving_scheme_new = pd.read_csv('./current_data/week_'+cur_week_str+'/receiving_scheme_pp_w'+cur_week_str+".csv")

rush_summ_conc_new = pd.read_csv('./current_data/week_'+cur_week_str+'/rush_summ_conc_pp_w'+cur_week_str+".csv")

block_summ_conc_new = pd.read_csv('./current_data/week_'+cur_week_str+'/block_summ_conc_pp_w'+cur_week_str+".csv")
offense_pass_blocking_new = pd.read_csv('./current_data/week_'+cur_week_str+'/offense_pass_blocking_pp_w'+cur_week_str+".csv")
offense_run_blocking_new = pd.read_csv('./current_data/week_'+cur_week_str+'/offense_run_blocking_pp_w'+cur_week_str+".csv")

def_summ_conc_new = pd.read_csv('./current_data/week_'+cur_week_str+'/def_summ_conc_pp_w'+cur_week_str+".csv")
pass_rush_summary_new = pd.read_csv('./current_data/week_'+cur_week_str+'/pass_rush_summary_pp_w'+cur_week_str+".csv")
run_defense_summary_new = pd.read_csv('./current_data/week_'+cur_week_str+'/run_defense_summary_pp_w'+cur_week_str+".csv")
defense_coverage_scheme_new = pd.read_csv('./current_data/week_'+cur_week_str+'/defense_coverage_scheme_pp_w'+cur_week_str+".csv")
defense_coverage_summary_new = pd.read_csv('./current_data/week_'+cur_week_str+'/defense_coverage_summary_pp_w'+cur_week_str+".csv")
slot_coverage_new = pd.read_csv('./current_data/week_'+cur_week_str+'/slot_coverage_pp_w'+cur_week_str+".csv")

st_kickers_new = pd.read_csv('./current_data/week_'+cur_week_str+'/st_kickers_pp_w'+cur_week_str+".csv")
st_punters_new = pd.read_csv('./current_data/week_'+cur_week_str+'/st_punters_no_inj_pp_w'+cur_week_str+".csv")


passing_depth_new['week'] = cur_week_str 
passing_allowed_pressure_new['week'] = cur_week_str 
passing_pressure_new['week'] = cur_week_str 
passing_concept_new['week'] = cur_week_str 
time_in_pocket_new['week'] = cur_week_str 
passing_summ_conc_new['week'] = cur_week_str 
rec_summ_conc_new['week'] = cur_week_str 
receiving_concept_new['week'] = cur_week_str
receiving_depth_new['week'] = cur_week_str 
receiving_scheme_new['week'] = cur_week_str 
rush_summ_conc_new['week'] = cur_week_str
block_summ_conc_new['week'] = cur_week_str 
offense_pass_blocking_new['week'] = cur_week_str 
offense_run_blocking_new['week'] = cur_week_str 
def_summ_conc_new['week'] = cur_week_str 
pass_rush_summary_new['week'] = cur_week_str 
run_defense_summary_new['week'] = cur_week_str 
defense_coverage_scheme_new['week'] = cur_week_str 
defense_coverage_summary_new['week'] = cur_week_str 
slot_coverage_new['week'] = cur_week_str 
st_kickers_new['week'] = cur_week_str 
st_punters_new['week'] = cur_week_str



In [79]:
####################################################################################
								###   add prefixes ###
####################################################################################	

def create_prefix(prefix=None, df=None):
    id = df[['p_id','player_team_id','unique_team_id','team_id_impute','player','numeric_id','position','team_name','year','week']]
    temp = df.drop(['p_id','player','player_team_id','unique_team_id','player','team_id_impute','numeric_id','position','team_name','unique_team_id','numeric_id','position','team_name','year','week','plyr_number'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id

def id_prefix(prefix=None, df=None):
    id = df[['p_id','player','player_team_id','team_id_impute','numeric_id','position','team_name','year','week']]
    temp = df.drop(['p_id','player','player_team_id','team_id_impute','numeric_id','position','team_name','year','week','plyr_number'], axis=1)
    temp = temp.add_prefix(prefix)
    id = pd.concat([id, temp], axis=1)
    return id

passing_summ_conc_new = id_prefix(prefix="pass_summary_", df=passing_summ_conc_new)
rush_summ_conc_new  = id_prefix(prefix="rush_summary_", df=rush_summ_conc_new)
rec_summ_conc_new  = id_prefix(prefix="rec_summary_", df=rec_summ_conc_new)
block_summ_conc_new  = id_prefix(prefix="block_summary_", df=block_summ_conc_new)
def_summ_conc_new  = id_prefix(prefix="def_summary_", df=def_summ_conc_new)
st_kickers_new  = id_prefix(prefix="kicking_", df=st_kickers_new)
st_punters_new  = id_prefix(prefix="punting_", df=st_punters_new)


passing_depth_new = create_prefix(prefix="pass_depth_", df=passing_depth_new)
passing_allowed_pressure_new = create_prefix(prefix="pressure_source_", df=passing_allowed_pressure_new)
passing_pressure_new = create_prefix(prefix="pass_under_pressure_", df=passing_pressure_new)
passing_concept_new = create_prefix(prefix="pass_concept_", df=passing_concept_new)
time_in_pocket_new = create_prefix(prefix="pass_time_", df=time_in_pocket_new)


receiving_concept_new = create_prefix(prefix="rec_concept_", df=receiving_concept_new)
receiving_depth_new = create_prefix(prefix="rec_depth_", df=receiving_depth_new)
receiving_scheme_new = create_prefix(prefix="rec_scheme_", df=receiving_scheme_new)

offense_pass_blocking_new = create_prefix(prefix="pass_block_", df=offense_pass_blocking_new)
offense_run_blocking_new = create_prefix(prefix="run_block_", df=offense_run_blocking_new)


pass_rush_summary_new = create_prefix(prefix="pass_rush_", df=pass_rush_summary_new)
run_defense_summary_new = create_prefix(prefix="run_defense_", df=run_defense_summary_new)
defense_coverage_scheme_new = create_prefix(prefix="def_coverage_scheme_", df=defense_coverage_scheme_new)
defense_coverage_summary_new = create_prefix(prefix="def_coverage_summary_", df=defense_coverage_summary_new)
slot_coverage_new= create_prefix(prefix="def_slot_coverage_", df=slot_coverage_new)


In [80]:
###Bring the historic and new player pool data together###

In [81]:
passing_depth = pd.concat([passing_depth, passing_depth_new], axis=0)
passing_allowed_pressure = pd.concat([passing_allowed_pressure, passing_allowed_pressure_new], axis=0)
passing_pressure = pd.concat([passing_pressure, passing_pressure_new], axis=0)
passing_concept = pd.concat([passing_concept_new, passing_concept_new], axis=0)
time_in_pocket = pd.concat([time_in_pocket, time_in_pocket_new], axis=0)
passing_summ_conc = pd.concat([passing_summ_conc, passing_summ_conc_new], axis=0)


rec_summ_conc = pd.concat([rec_summ_conc, rec_summ_conc_new], axis=0)
receiving_concept = pd.concat([receiving_concept, receiving_concept_new], axis=0)
receiving_depth = pd.concat([receiving_depth, receiving_depth_new], axis=0)
receiving_scheme = pd.concat([receiving_scheme, receiving_scheme_new], axis=0)

rush_summ_conc = pd.concat([rush_summ_conc, rush_summ_conc_new], axis=0)

block_summ_conc = pd.concat([block_summ_conc, block_summ_conc_new], axis=0)
offense_pass_blocking = pd.concat([offense_pass_blocking, offense_pass_blocking_new], axis=0)
offense_run_blocking = pd.concat([offense_run_blocking, offense_run_blocking_new], axis=0)

def_summ_conc = pd.concat([def_summ_conc, def_summ_conc_new], axis=0)
pass_rush_summary = pd.concat([pass_rush_summary, pass_rush_summary_new], axis=0)
run_defense_summary = pd.concat([run_defense_summary, run_defense_summary_new], axis=0)
defense_coverage_scheme = pd.concat([defense_coverage_scheme, defense_coverage_scheme_new], axis=0)
defense_coverage_summary = pd.concat([defense_coverage_summary, defense_coverage_summary_new], axis=0)
slot_coverage = pd.concat([slot_coverage, slot_coverage_new], axis=0)

st_kickers = pd.concat([st_kickers, st_kickers_new], axis=0)
st_punters = pd.concat([st_punters, st_punters_new], axis=0)


### after the concat cell ###
passing_depth.drop_duplicates(subset='p_id', inplace=True)
passing_allowed_pressure.drop_duplicates(subset='p_id', inplace=True)
passing_pressure.drop_duplicates(subset='p_id', inplace=True)
passing_concept.drop_duplicates(subset='p_id', inplace=True)
time_in_pocket.drop_duplicates(subset='p_id', inplace=True)
passing_summ_conc.drop_duplicates(subset='p_id', inplace=True)


rec_summ_conc.drop_duplicates(subset='p_id', inplace=True)
receiving_concept.drop_duplicates(subset='p_id', inplace=True)
receiving_depth.drop_duplicates(subset='p_id', inplace=True)
receiving_scheme.drop_duplicates(subset='p_id', inplace=True)

rush_summ_conc.drop_duplicates(subset='p_id', inplace=True)

block_summ_conc.drop_duplicates(subset='p_id', inplace=True)
offense_pass_blocking.drop_duplicates(subset='p_id', inplace=True)
offense_run_blocking.drop_duplicates(subset='p_id', inplace=True)

def_summ_conc.drop_duplicates(subset='p_id', inplace=True)
pass_rush_summary.drop_duplicates(subset='p_id', inplace=True)
run_defense_summary.drop_duplicates(subset='p_id', inplace=True)
defense_coverage_scheme.drop_duplicates(subset='p_id', inplace=True)
defense_coverage_summary.drop_duplicates(subset='p_id', inplace=True)
slot_coverage.drop_duplicates(subset='p_id', inplace=True)

st_kickers.drop_duplicates(subset='p_id', inplace=True)
st_punters.drop_duplicates(subset='p_id', inplace=True)


In [82]:
rush_summ_conc_temp = rush_summ_conc[rush_summ_conc['position'] == 'hb']
rush_summ_conc.head()

,p_id,player,player_team_id,unique_team_id,team_id_impute,numeric_id,position,team_name,year,week,...,rush_summary_targets,rush_summary_total_touches,rush_summary_touchdowns,rush_summary_yards,rush_summary_yards_after_contact,rush_summary_yco_attempt,rush_summary_ypa,rush_summary_yprr,rush_summary_zone_attempts,rush_summary_unique_team_id
0,joemixon_cin_2021_1,joemixon,joemixon_cin_2021,cin_2021_1,cin_2021,11803,hb,cin,2021,1,...,4,33,1,127,78.0,2.69,4.4,1.28,18,NaN
1,markingramii_hou_2021_1,markingramii,markingramii_hou_2021,hou_2021_1,hou_2021,6180,hb,hou,2021,1,...,1,26,1,85,70.0,2.69,3.3,0.00,8,NaN
2,damienharris_ne_2021_1,damienharris,damienharris_ne_2021,ne_2021_1,ne_2021,35134,hb,ne,2021,1,...,3,25,0,100,68.0,2.96,4.3,1.70,12,NaN
3,christianmccaffrey_car_2021_1,christianmccaffrey,christianmccaffrey_car_2021,car_2021_1,car_2021,11763,hb,car,2021,1,...,9,30,0,98,58.0,2.76,4.7,4.05,15,NaN
4,dalvincook_min_2021_1,dalvincook,dalvincook_min_2021,min_2021_1,min_2021,11796,hb,min,2021,1,...,7,26,1,61,43.0,2.15,3.1,1.65,17,NaN


In [83]:
rush_summ_conc=rush_summ_conc_temp

In [84]:
%%time

def rolling(data=None, roll_value=None, roll_type=None):
    
    """
        Args:
        data: input pandas dataframe to be rolled
        roll_value: input the number, default is three ## we will need to modify the function if we want more ##
        roll_type: 'mean','std', or 'var' are the only options at the point
        ## assign mean for a given team & year as opposed to the entire dataset
   
    """
    
    data = data.sort_values(by=["player","team_name","year","week"], ascending=[True, True, True, True])
    data['week']=data['week'].apply(str)
    data['year']=data['year'].apply(str)
    num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    ids = pd.DataFrame(data.select_dtypes(exclude=[np.number])).reset_index(drop=True)
   
    if roll_type == 'mean':
        #roll5 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        #roll4 = data.groupby(data['player_id'])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll3 = data.groupby(data["player_team_id"])[num_cols].apply(lambda x : x.shift().rolling(roll_value).mean())
        roll2 = data.groupby(data["player_team_id"])[num_cols].apply(lambda x : x.shift().rolling(roll_value-1).mean())
        roll1 = data.groupby(data["player_team_id"])[num_cols].apply(lambda x : x.shift().rolling(roll_value-2).mean())
        roll3 = pd.DataFrame(roll3.combine_first(roll2).combine_first(roll1)).reset_index(drop=True)
        df = pd.concat([ids, roll3], axis=1)
    return df
   
passing_depth_roll = rolling(data=passing_depth, roll_value=3, roll_type='mean')
passing_allowed_pressure_roll = rolling(data=passing_allowed_pressure, roll_value=3, roll_type='mean')
passing_pressure_roll = rolling(data=passing_pressure, roll_value=3, roll_type='mean')
passing_concept_roll = rolling(data=passing_concept, roll_value=3, roll_type='mean')
time_in_pocket_roll = rolling(data=time_in_pocket, roll_value=3, roll_type='mean')
passing_summ_conc_roll = rolling(data=passing_summ_conc, roll_value=3, roll_type='mean')


rec_summ_conc_roll = rolling(data=rec_summ_conc, roll_value=3, roll_type='mean')
receiving_concept_roll =rolling(data=receiving_concept, roll_value=3, roll_type='mean')
receiving_depth_roll = rolling(data=receiving_depth, roll_value=3, roll_type='mean')
receiving_scheme_roll = rolling(data=receiving_scheme, roll_value=3, roll_type='mean')

rush_summ_conc_roll = rolling(data=rush_summ_conc, roll_value=3, roll_type='mean')

block_summ_conc_roll = rolling(data=block_summ_conc, roll_value=3, roll_type='mean')
offense_pass_blocking_roll = rolling(data=offense_pass_blocking, roll_value=3, roll_type='mean')
offense_run_blocking_roll = rolling(data=offense_run_blocking, roll_value=3, roll_type='mean')

def_summ_conc_roll = rolling(data=def_summ_conc, roll_value=3, roll_type='mean')
pass_rush_summary_roll = rolling(data=pass_rush_summary, roll_value=3, roll_type='mean')
run_defense_summary_roll = rolling(data=run_defense_summary, roll_value=3, roll_type='mean')
defense_coverage_scheme_roll = rolling(data=defense_coverage_scheme, roll_value=3, roll_type='mean')
defense_coverage_summary_roll = rolling(data=defense_coverage_summary, roll_value=3, roll_type='mean')
slot_coverage_roll = rolling(data=slot_coverage, roll_value=3, roll_type='mean')

st_kickers_roll = rolling(data=st_kickers, roll_value=3, roll_type='mean')
st_punters_roll = rolling(data=st_punters, roll_value=3, roll_type='mean')

CPU times: user 6min 59s, sys: 443 ms, total: 6min 59s
Wall time: 6min 59s


In [88]:
rush_summ_conc_roll = rolling(data=rush_summ_conc, roll_value=3, roll_type='mean')
rush_summ_conc_roll.tail(n=25)

,p_id,player,player_team_id,unique_team_id,team_id_impute,position,team_name,year,week,rush_summary_unique_team_id,...,rush_summary_scrambles,rush_summary_targets,rush_summary_total_touches,rush_summary_touchdowns,rush_summary_yards,rush_summary_yards_after_contact,rush_summary_yco_attempt,rush_summary_ypa,rush_summary_yprr,rush_summary_zone_attempts
10264,zacstacy_lar_2014_1,zacstacy,zacstacy_lar_2014,lar_2014_1,lar_2014,hb,lar,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10265,zacstacy_lar_2014_2,zacstacy,zacstacy_lar_2014,lar_2014_2,lar_2014,hb,lar,2014,2,NaN,...,0.0,2.000000,12.000000,0.000000,43.000000,22.000000,2.000000,3.900000,0.730000,6.000000
10266,zacstacy_lar_2014_3,zacstacy,zacstacy_lar_2014,lar_2014_3,lar_2014,hb,lar,2014,3,NaN,...,0.0,1.000000,15.500000,0.500000,57.000000,28.500000,1.920000,3.800000,0.365000,10.000000
10267,zacstacy_lar_2014_5,zacstacy,zacstacy_lar_2014,lar_2014_5,lar_2014,hb,lar,2014,5,NaN,...,0.0,2.666667,16.000000,0.333333,60.333333,35.333333,2.640000,4.400000,1.243333,8.333333
10268,zacstacy_lar_2014_6,zacstacy,zacstacy_lar_2014,lar_2014_6,lar_2014,hb,lar,2014,6,NaN,...,0.0,3.333333,17.000000,0.333333,61.666667,35.333333,2.640000,4.533333,1.750000,8.666667
10269,zacstacy_lar_2014_8,zacstacy,zacstacy_lar_2014,lar_2014_8,lar_2014,hb,lar,2014,8,NaN,...,0.0,4.333333,14.000000,0.000000,43.666667,27.333333,2.486667,4.000000,2.266667,5.333333
10270,zacstacy_lar_2014_13,zacstacy,zacstacy_lar_2014,lar_2014_13,lar_2014,hb,lar,2014,13,NaN,...,0.0,3.666667,11.000000,0.000000,27.000000,14.666667,1.860000,3.266667,2.006667,5.333333
10271,zacstacy_lar_2014_16,zacstacy,zacstacy_lar_2014,lar_2014_16,lar_2014,hb,lar,2014,16,NaN,...,0.0,2.333333,8.000000,0.000000,20.333333,12.666667,2.083333,3.333333,1.256667,5.000000
10272,zacstacy_lar_2014_17,zacstacy,zacstacy_lar_2014,lar_2014_17,lar_2014,hb,lar,2014,17,NaN,...,0.0,1.333333,5.333333,0.000000,16.666667,10.666667,2.456667,3.633333,0.740000,4.333333
10273,zacstacy_nyj_2015_3,zacstacy,zacstacy_nyj_2015,nyj_2015_3,nyj_2015,hb,nyj,2015,3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
def filter_fillna(df=None, position=None, min_Var=None):
    sub= df[df['position'].str.match(position)]
    sub_limit = sub[(sub[min_Var] <=5) & (sub[min_Var] >=1)]
    buckup_df = pd.DataFrame(sub_limit.median()).T
    num_cols = sub.select_dtypes(include=[np.number]).columns.tolist()
    msk = sub.isnull()
    tmp = sub[num_cols].mask(msk, buckup_df[num_cols])
    tmp = np.where(msk[num_cols], buckup_df[num_cols], tmp[num_cols])
    tmp = pd.DataFrame(tmp, columns=buckup_df.columns)
    ids = pd.DataFrame(sub.select_dtypes(exclude=[np.number])).reset_index(drop=True)
    mrg = pd.concat([ids, tmp], axis=1)
    return mrg
    

In [91]:
rush_summ_hbfb = filter_fillna(df=rush_summ_conc_roll, position='hb|fb', min_Var='rush_summary_attempts')
#rush_summ_qb = filter_fillna(df=rush_summ_conc_roll, position='qb')
rush_summ_wr = filter_fillna(df=rush_summ_conc_roll, position='wr', min_Var='rush_summary_attempts')
rush_summ_conc_roll = pd.concat([rush_summ_hbfb,rush_summ_wr], axis=0)

/home/tomb/anaconda3/envs/cloak/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  after removing the cwd from sys.path.


ValueError: Shape of passed values is (10289, 44), indices imply (10289, 46)

In [92]:
## make sure we aren't weighting w/a 0 value (non-designed runs are cancelled ##
def rush_att(nData, var=None):
    if nData[var] == 0:
        return 1
    else:
        return nData[var]

rush_summ_conc_roll['rush_summary_attempts'] = rush_summ_conc_roll.apply(lambda df: rush_att(df, var='rush_summary_attempts'), axis=1)   


def weighted(nData, snap_Var='rush_summary_attempts'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)


rb_stats = rush_summ_conc_roll.groupby('unique_team_id').apply(weighted).reset_index()



,unique_team_id,numeric_id,rush_summary_attempts,rush_summary_avoided_tackles,rush_summary_breakaway_attempts,rush_summary_breakaway_percent,rush_summary_breakaway_yards,rush_summary_declined_penalties,rush_summary_designed_yards,rush_summary_drops,...,rush_summary_scrambles,rush_summary_targets,rush_summary_total_touches,rush_summary_touchdowns,rush_summary_yards,rush_summary_yards_after_contact,rush_summary_yco_attempt,rush_summary_ypa,rush_summary_yprr,rush_summary_zone_attempts
4283,was_2022_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4284,was_2022_2,66225.941176,12.058824,3.470588,0.0,0.0,0.0,0.0,49.176471,0.000000,...,0.0,7.117647,18.352941,0.000000,49.176471,40.235294,3.059412,3.852941,2.785882,9.588235
4285,was_2022_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4286,was_2022_4,62572.942308,10.833333,1.852564,0.0,0.0,0.0,0.0,33.583333,0.628205,...,0.0,4.371795,14.320513,0.512821,33.583333,27.506410,2.537372,3.215385,1.280641,7.583333
4287,was_2022_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
passing_depth_roll = filter_fillna(df=passing_depth_roll, min_Var='pass_depth_dropbacks')
passing_allowed_pressure_roll = filter_fillna(df=passing_allowed_pressure_roll, min_Var='pressure_source_allowed_pressure_dropbacks')
passing_pressure_roll = filter_fillna(df=passing_pressure_roll, min_Var='pass_under_pressure_base_dropbacks')
passing_concept_roll = filter_fillna(df=passing_concept_roll, min_Var='pass_concept_dropbacks')
time_in_pocket_roll = filter_fillna(df=time_in_pocket_roll, min_Var='pass_time_dropbacks')
passing_summ_conc_roll = filter_fillna_qbs(df=passing_summ_conc_roll, min_Var='pass_summary_dropbacks')


def pass_att(nData, var=None):
    if nData[var] == 0:
        return 1
    else:
        return nData[var]
passing_summ_conc_roll['pass_summary_dropbacks'] = passing_summ_conc_roll.apply(lambda df: pass_att(df, var='pass_summary_dropbacks'), axis=1)
passing_depth_roll['pass_depth_dropbacks'] = passing_depth_roll.apply(lambda df: pass_att(df, var='pass_depth_dropbacks'), axis=1)  
passing_pressure_roll['pass_under_pressure_base_dropbacks'] = passing_pressure_roll.apply(lambda df: pass_att(df, var='pass_under_pressure_base_dropbacks'), axis=1)  
passing_allowed_pressure_roll['pressure_source_allowed_pressure_dropbacks'] = passing_allowed_pressure_roll.apply(lambda df: pass_att(df, var='pressure_source_allowed_pressure_dropbacks'), axis=1)  
passing_concept_roll['pass_concept_dropbacks'] = passing_summ_conc_roll.apply(lambda df: pass_att(df, var='pass_concept_dropbacks'), axis=1)  
time_in_pocket_roll['pass_time_dropbacks'] = time_in_pocket_roll.apply(lambda df: pass_att(df, var='pass_time_dropbacks'), axis=1)     


def weighted(nData, snap_Var='pass_summary_dropbacks'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
qb_stats = passing_summ_conc_roll.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='pass_depth_dropbacks'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
pass_depth_stats = passing_depth_roll.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='pressure_source_allowed_pressure_dropbacks'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
passing_allowed_pressure_stats = passing_allowed_pressure_roll.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='pass_under_pressure_base_dropbacks'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
passing_pressure_stats = passing_pressure_roll.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='pass_concept_dropbacks'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
passing_concept_stats = passing_concept_roll.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='pass_time_dropbacks'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
time_in_pocke_stats = time_in_pocket_roll.groupby('unique_team_id').apply(weighted).reset_index()

TypeError: first argument must be string or compiled pattern

In [ ]:
## make sure we aren't weighting w/a 0 value (non-designed runs are cancelled ##
def rec_att(nData, var=None):
    if nData[var] == 0:
        return 1
    else:
        return nData[var]

rec_summ_conc_roll['rec_summary_targets'] = rec_summ_conc_roll.apply(lambda df: rec_att(df, var='rec_summary_targets'), axis=1)   
receiving_concept_roll['rec_concept_targets'] = receiving_concept_roll.apply(lambda df: rec_att(df, var='rec_concept_targets'), axis=1) 
receiving_depth_roll['rec_depth_targets'] = receiving_depth_roll.apply(lambda df: rec_att(df, var='rec_depth_targets'), axis=1) 
receiving_scheme_roll['rec_scheme_targets'] = receiving_scheme_roll.apply(lambda df: rec_att(df, var='rec_scheme_targets'), axis=1) 


rec_summ_conc = rolling(data=rec_summ_conc, roll_value=3, roll_type='mean')
receiving_concept =rolling(data=receiving_concept, roll_value=3, roll_type='mean')
receiving_depth = rolling(data=receiving_depth, roll_value=3, roll_type='mean')
receiving_scheme = rolling(data=receiving_scheme, roll_value=3, roll_type='mean')


def weighted(nData, snap_Var='rec_summary_targets'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)

rec_stats = rec_summ_conc_roll.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='rec_concept_targets'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
	
receiving_concept = receiving_concept.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='rec_depth_targets'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
	
receiving_depth = receiving_depth.groupby('unique_team_id').apply(weighted).reset_index()

def weighted(nData, snap_Var='rec_scheme_targets'):
    data_cols = nData.select_dtypes(include=[np.number])
    num_cols = data_cols[data_cols.columns.drop(list(data_cols.filter(regex='player_game_count|player_id|plyr_number|week|year|team_id')))].columns.tolist()
    return pd.Series(np.average(nData[num_cols], weights=nData[snap_Var], axis=0), num_cols)
	
receiving_scheme = receiving_scheme.groupby('unique_team_id').apply(weighted).reset_index()